# Comparing Reasoning Paradigms

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/neurosym-kg/blob/main/notebooks/02_comparing_reasoners.ipynb)

This notebook compares different neuro-symbolic reasoning paradigms:

| Reasoner | Paper | Approach |
|----------|-------|----------|
| **Think-on-Graph (ToG)** | ICLR 2024 | LLM as agent with beam search |
| **Reasoning on Graphs (RoG)** | ICLR 2024 | Plan-based faithful reasoning |
| **GraphRAG** | Microsoft 2024 | Community-based retrieval |
| **SubgraphRAG** | 2024 | Flexible subgraph retrieval |

## 1. Setup

In [ ]:
# Install dependencies (run once)
!pip install -q pydantic pydantic-settings httpx tenacity networkx numpy tqdm

# Clone repo if needed (uncomment)
# !git clone https://github.com/your-org/neurosym-kg.git
# import sys; sys.path.insert(0, 'neurosym-kg')

In [ ]:
import sys
import time
from typing import Dict, List

# Import framework
from neurosym_kg import (
    InMemoryKG,
    Triple,
    MockLLMBackend,
)
from neurosym_kg.reasoners import (
    ThinkOnGraphReasoner,
    ReasoningOnGraphs,
    GraphRAGReasoner,
    SubgraphRAGReasoner,
)
from neurosym_kg.evaluation import exact_match, f1_score

print("✅ All imports successful!")

## 2. Create Test Knowledge Graph

We'll use a movie knowledge graph for this comparison.

In [ ]:
def create_movie_kg() -> InMemoryKG:
    """Create a movie knowledge graph for testing."""
    kg = InMemoryKG(name="Movies KG")
    
    kg.add_triples([
        # Inception
        Triple("Inception", "director", "Christopher_Nolan"),
        Triple("Inception", "year", "2010"),
        Triple("Inception", "genre", "Science_Fiction"),
        Triple("Inception", "starring", "Leonardo_DiCaprio"),
        Triple("Inception", "starring", "Tom_Hardy"),
        Triple("Inception", "starring", "Ellen_Page"),
        Triple("Inception", "budget", "160_million_USD"),
        
        # The Dark Knight
        Triple("The_Dark_Knight", "director", "Christopher_Nolan"),
        Triple("The_Dark_Knight", "year", "2008"),
        Triple("The_Dark_Knight", "genre", "Action"),
        Triple("The_Dark_Knight", "starring", "Christian_Bale"),
        Triple("The_Dark_Knight", "starring", "Heath_Ledger"),
        
        # Interstellar
        Triple("Interstellar", "director", "Christopher_Nolan"),
        Triple("Interstellar", "year", "2014"),
        Triple("Interstellar", "genre", "Science_Fiction"),
        Triple("Interstellar", "starring", "Matthew_McConaughey"),
        
        # Director info
        Triple("Christopher_Nolan", "born_in", "London"),
        Triple("Christopher_Nolan", "nationality", "British"),
        Triple("Christopher_Nolan", "occupation", "Director"),
        Triple("Christopher_Nolan", "birth_year", "1970"),
        
        # Actor info
        Triple("Leonardo_DiCaprio", "born_in", "Los_Angeles"),
        Triple("Leonardo_DiCaprio", "occupation", "Actor"),
        Triple("Leonardo_DiCaprio", "award", "Academy_Award_2016"),
        
        Triple("Tom_Hardy", "born_in", "London"),
        Triple("Tom_Hardy", "occupation", "Actor"),
        
        # Titanic (different director)
        Triple("Titanic", "director", "James_Cameron"),
        Triple("Titanic", "year", "1997"),
        Triple("Titanic", "starring", "Leonardo_DiCaprio"),
        Triple("Titanic", "starring", "Kate_Winslet"),
        Triple("Titanic", "genre", "Drama"),
        
        # Location hierarchy
        Triple("London", "located_in", "United_Kingdom"),
        Triple("Los_Angeles", "located_in", "California"),
        Triple("California", "located_in", "United_States"),
    ])
    
    return kg

kg = create_movie_kg()
print(kg.summary())

## 3. Create Test Questions

We'll test with questions of varying complexity.

In [ ]:
# Test questions with ground truth answers
TEST_QUESTIONS = [
    # 1-hop questions (direct lookup)
    {
        "question": "Who directed Inception?",
        "answers": ["Christopher Nolan", "Christopher_Nolan"],
        "type": "1-hop",
    },
    {
        "question": "What year was The Dark Knight released?",
        "answers": ["2008"],
        "type": "1-hop",
    },
    {
        "question": "What genre is Interstellar?",
        "answers": ["Science Fiction", "Science_Fiction", "Sci-Fi"],
        "type": "1-hop",
    },
    
    # 2-hop questions (require traversal)
    {
        "question": "Where was the director of Inception born?",
        "answers": ["London"],
        "type": "2-hop",
    },
    {
        "question": "What is the nationality of the director of The Dark Knight?",
        "answers": ["British"],
        "type": "2-hop",
    },
    
    # Aggregation questions
    {
        "question": "What movies did Christopher Nolan direct?",
        "answers": ["Inception", "The Dark Knight", "Interstellar"],
        "type": "aggregation",
    },
]

print(f"Created {len(TEST_QUESTIONS)} test questions:")
for i, q in enumerate(TEST_QUESTIONS, 1):
    print(f"  {i}. [{q['type']}] {q['question']}")

## 4. Create Mock LLM

For reproducible comparisons, we use a mock LLM with predefined responses.

In [ ]:
def create_mock_llm() -> MockLLMBackend:
    """Create a mock LLM with responses for our movie questions."""
    llm = MockLLMBackend(default_response="Unknown")
    
    # Entity extraction patterns
    llm.add_response(r".*Extract.*Inception.*", "Inception")
    llm.add_response(r".*Extract.*Dark Knight.*", "The_Dark_Knight")
    llm.add_response(r".*Extract.*Interstellar.*", "Interstellar")
    llm.add_response(r".*Extract.*Nolan.*", "Christopher_Nolan")
    llm.add_response(r".*Extract.*director.*Inception.*", "Inception")
    llm.add_response(r".*Extract.*movies.*Nolan.*", "Christopher_Nolan")
    
    # Relation selection
    llm.add_response(r".*relations.*director.*", "director")
    llm.add_response(r".*relations.*year.*", "year")
    llm.add_response(r".*relations.*genre.*", "genre")
    llm.add_response(r".*relations.*born.*", "born_in")
    llm.add_response(r".*relations.*nationality.*", "nationality")
    llm.add_response(r".*relations.*movie.*", "director")
    
    # Plan generation for RoG
    llm.add_response(r".*relation paths.*director.*born.*", "director -> born_in")
    llm.add_response(r".*relation paths.*director.*nationality.*", "director -> nationality")
    llm.add_response(r".*relation paths.*director.*", "director")
    
    # Reasoning checks
    llm.add_response(r".*enough information.*", "YES - Found relevant facts.")
    
    # Answer generation
    llm.add_response(r".*answer.*director.*Inception.*", "Christopher Nolan directed Inception.")
    llm.add_response(r".*answer.*year.*Dark Knight.*", "The Dark Knight was released in 2008.")
    llm.add_response(r".*answer.*genre.*Interstellar.*", "Interstellar is a Science Fiction film.")
    llm.add_response(r".*answer.*born.*director.*Inception.*", "Christopher Nolan was born in London.")
    llm.add_response(r".*answer.*nationality.*director.*", "Christopher Nolan is British.")
    llm.add_response(r".*answer.*movies.*Nolan.*", "Christopher Nolan directed Inception, The Dark Knight, and Interstellar.")
    
    return llm

print("✅ Mock LLM factory created")

## 5. Initialize All Reasoners

In [ ]:
# Create all reasoners
reasoners = {
    "Think-on-Graph (ToG)": ThinkOnGraphReasoner(
        kg=kg,
        llm=create_mock_llm(),
        max_depth=3,
        beam_width=5,
        verbose=False,
    ),
    "Reasoning on Graphs (RoG)": ReasoningOnGraphs(
        kg=kg,
        llm=create_mock_llm(),
        max_path_length=3,
        num_plans=3,
        verbose=False,
    ),
    "GraphRAG": GraphRAGReasoner(
        kg=kg,
        llm=create_mock_llm(),
        verbose=False,
    ),
    "SubgraphRAG": SubgraphRAGReasoner(
        kg=kg,
        llm=create_mock_llm(),
        subgraph_size=30,
        verbose=False,
    ),
}

print(f"✅ Initialized {len(reasoners)} reasoners:")
for name in reasoners:
    print(f"   • {name}")

## 6. Run Comparison

In [ ]:
def evaluate_reasoner(reasoner, questions: List[Dict]) -> Dict:
    """Evaluate a reasoner on a set of questions."""
    results = {
        "correct": 0,
        "total": len(questions),
        "total_latency_ms": 0,
        "details": [],
    }
    
    for q in questions:
        try:
            result = reasoner.reason(q["question"])
            
            # Calculate metrics
            em = exact_match(result.answer, q["answers"])
            f1 = f1_score(result.answer, q["answers"])
            
            results["correct"] += em
            results["total_latency_ms"] += result.latency_ms
            
            results["details"].append({
                "question": q["question"],
                "answer": result.answer,
                "expected": q["answers"],
                "exact_match": em,
                "f1": f1,
                "latency_ms": result.latency_ms,
            })
        except Exception as e:
            results["details"].append({
                "question": q["question"],
                "error": str(e),
            })
    
    results["accuracy"] = results["correct"] / results["total"]
    results["avg_latency_ms"] = results["total_latency_ms"] / results["total"]
    
    return results

In [ ]:
# Run evaluation for all reasoners
print("Running evaluation...\n")
print("=" * 70)

all_results = {}

for name, reasoner in reasoners.items():
    print(f"\n📊 Evaluating: {name}")
    print("-" * 40)
    
    results = evaluate_reasoner(reasoner, TEST_QUESTIONS)
    all_results[name] = results
    
    print(f"   Accuracy: {results['accuracy']:.1%}")
    print(f"   Avg Latency: {results['avg_latency_ms']:.0f}ms")

print("\n" + "=" * 70)

## 7. Results Comparison

In [ ]:
# Summary table
print("\n" + "=" * 70)
print("                    COMPARISON RESULTS")
print("=" * 70)
print(f"{'Reasoner':<30} {'Accuracy':>12} {'Avg Latency':>15}")
print("-" * 70)

for name, results in all_results.items():
    print(f"{name:<30} {results['accuracy']:>11.1%} {results['avg_latency_ms']:>12.0f}ms")

print("=" * 70)

In [ ]:
# Detailed per-question results
print("\n📋 Detailed Results by Question")
print("=" * 80)

for i, q in enumerate(TEST_QUESTIONS):
    print(f"\nQ{i+1}: {q['question']}")
    print(f"    Expected: {q['answers']}")
    print(f"    Type: {q['type']}")
    print()
    
    for name, results in all_results.items():
        detail = results["details"][i]
        if "error" in detail:
            print(f"    {name:<25}: ❌ Error - {detail['error'][:30]}")
        else:
            status = "✅" if detail["exact_match"] else "❌"
            print(f"    {name:<25}: {status} {detail['answer'][:40]}")

## 8. Visualize Results

In [ ]:
# Simple ASCII bar chart
print("\n📊 Accuracy Comparison")
print("=" * 50)

max_bar = 40
for name, results in all_results.items():
    acc = results['accuracy']
    bar_len = int(acc * max_bar)
    bar = "█" * bar_len + "░" * (max_bar - bar_len)
    print(f"{name:<25}")
    print(f"  [{bar}] {acc:.1%}")

## 9. Analysis by Question Type

In [ ]:
# Analyze performance by question type
question_types = set(q["type"] for q in TEST_QUESTIONS)

print("\n📈 Performance by Question Type")
print("=" * 60)

for qtype in question_types:
    print(f"\n{qtype.upper()}:")
    
    # Get indices for this question type
    indices = [i for i, q in enumerate(TEST_QUESTIONS) if q["type"] == qtype]
    
    for name, results in all_results.items():
        correct = sum(1 for i in indices if results["details"][i].get("exact_match", 0))
        total = len(indices)
        acc = correct / total if total > 0 else 0
        print(f"  {name:<30}: {correct}/{total} ({acc:.0%})")

## 10. Using with Real LLM

To compare with a real LLM (better results, costs money):

In [ ]:
# Uncomment to use with OpenAI
"""
from neurosym_kg import OpenAIBackend

real_llm = OpenAIBackend(model="gpt-4o-mini")

real_reasoners = {
    "ToG + GPT-4o-mini": ThinkOnGraphReasoner(kg=kg, llm=real_llm),
    "RoG + GPT-4o-mini": ReasoningOnGraphs(kg=kg, llm=real_llm),
}

for name, reasoner in real_reasoners.items():
    print(f"\nTesting {name}...")
    result = reasoner.reason("Who directed Inception?")
    print(f"Answer: {result.answer}")
"""
print("💡 Uncomment the code above to compare with real LLM")

## 🎯 Key Takeaways

1. **Think-on-Graph (ToG)**: Best for exploratory queries, finds multiple reasoning paths
2. **Reasoning on Graphs (RoG)**: Best for structured multi-hop queries, provides faithful paths
3. **GraphRAG**: Best for global/aggregation queries over large KGs
4. **SubgraphRAG**: Good balance, flexible subgraph sizing

### When to Use Each:

| Use Case | Recommended Reasoner |
|----------|----------------------|
| Simple 1-hop queries | Any (SubgraphRAG fastest) |
| Multi-hop reasoning | ToG or RoG |
| Aggregation queries | GraphRAG |
| Limited compute | SubgraphRAG |
| Explainability needed | RoG (faithful paths) |